<a href="https://colab.research.google.com/github/arkhanks/Project_4_Machine_Learning/blob/main/Amyworkingupcolablast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [86]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RestaurantInspections").getOrCreate()

In [87]:
# Specify the file path to your local CSV file
file_path = "/content/ARCHIVED__Restaurant_Inspection_Scores__2016-2019__20250310.csv"  # Replace with your actual file path

# Read CSV file into Spark DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame to verify it loaded correctly
df.show()

+-----------+--------------------+--------------------+-------------+--------------+--------------------+-----------------+------------------+--------------------+---------------------+---------------+--------------------+----------------+--------------------+--------------------+---------------------+-------------+-------------+---------------------+------------------------+----------------------------+----------------------+
|business_id|       business_name|    business_address|business_city|business_state|business_postal_code|business_latitude|business_longitude|   business_location|business_phone_number|  inspection_id|     inspection_date|inspection_score|     inspection_type|        violation_id|violation_description|risk_category|Neighborhoods|SF Find Neighborhoods|Current Police Districts|Current Supervisor Districts|Analysis Neighborhoods|
+-----------+--------------------+--------------------+-------------+--------------+--------------------+-----------------+---------------

In [88]:
# Get the number of rows
num_rows = df.count()

# Print the result
print(f"Number of rows in the DataFrame: {num_rows}")

Number of rows in the DataFrame: 53973


In [89]:
df.printSchema()

root
 |-- business_id: integer (nullable = true)
 |-- business_name: string (nullable = true)
 |-- business_address: string (nullable = true)
 |-- business_city: string (nullable = true)
 |-- business_state: string (nullable = true)
 |-- business_postal_code: string (nullable = true)
 |-- business_latitude: double (nullable = true)
 |-- business_longitude: double (nullable = true)
 |-- business_location: string (nullable = true)
 |-- business_phone_number: long (nullable = true)
 |-- inspection_id: string (nullable = true)
 |-- inspection_date: string (nullable = true)
 |-- inspection_score: integer (nullable = true)
 |-- inspection_type: string (nullable = true)
 |-- violation_id: string (nullable = true)
 |-- violation_description: string (nullable = true)
 |-- risk_category: string (nullable = true)
 |-- Neighborhoods: integer (nullable = true)
 |-- SF Find Neighborhoods: integer (nullable = true)
 |-- Current Police Districts: integer (nullable = true)
 |-- Current Supervisor Distr

In [90]:
from pyspark.sql import functions as F

# Get the count of null values for each column to assist in determining which columns to drop before using dropna
null_counts = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Show the result
null_counts.show()

+-----------+-------------+----------------+-------------+--------------+--------------------+-----------------+------------------+-----------------+---------------------+-------------+---------------+----------------+---------------+------------+---------------------+-------------+-------------+---------------------+------------------------+----------------------------+----------------------+
|business_id|business_name|business_address|business_city|business_state|business_postal_code|business_latitude|business_longitude|business_location|business_phone_number|inspection_id|inspection_date|inspection_score|inspection_type|violation_id|violation_description|risk_category|Neighborhoods|SF Find Neighborhoods|Current Police Districts|Current Supervisor Districts|Analysis Neighborhoods|
+-----------+-------------+----------------+-------------+--------------+--------------------+-----------------+------------------+-----------------+---------------------+-------------+---------------+-----

--business_name: removed because business_id provides the required unique values and is a numerical value which will work better for an ML model.\
--business_address: Removed because we didn't want to go as granular as specific addresses.\
--business_city: removed because all data comes from San Francisco City\
--business_state: removed because all data comes from the State of California\
--business_postal_code: Kept so we can see if postal code area has any part in predicting inspection scores.\
--business_latitude and business_longitude: removed both because they had over 26,000 null values and would greatly decrease our data.\
--business_location: removed because wasn't pertinent\
--business_phone_number: removed because it didn't fit the scope of our project\
--inspection_id\
--inspection_date\
--inspection_score\
--inspection_type\
--violation_id\
--violation_description\
--risk_category\
--Neighborhoods, SF Find Neighborhoods, Current Police Districts, Current Supervisor Districts, Analysis Neighborhoods: all removed because they had over 26000 null values and there wasn't any reference to what any of them meant

In [139]:
columns_to_remove = ["business_name", "business_city", "business_address","business_state", "business_latitude", "business_longitude", "business_phone_number",
                     'business_location', 'inspection_type', 'Neighborhoods', 'SF Find Neighborhoods',
                     'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods']
# Get the list of existing columns to remove (checking if they exist in the DataFrame)
existing_columns_to_remove = [col for col in columns_to_remove if col in df.columns]

# Drop the columns that exist in the DataFrame
prepared_df = df.drop(*existing_columns_to_remove)
prepared_df.show()

+-----------+--------------------+---------------+--------------------+----------------+--------------------+---------------------+-------------+
|business_id|business_postal_code|  inspection_id|     inspection_date|inspection_score|        violation_id|violation_description|risk_category|
+-----------+--------------------+---------------+--------------------+----------------+--------------------+---------------------+-------------+
|      85936|               94108| 85936_20170925|09/25/2017 12:00:...|             100|                NULL|                 NULL|         NULL|
|       5827|               94134|  5827_20170224|02/24/2017 12:00:...|            NULL|                NULL|                 NULL|         NULL|
|      94910|               94109| 94910_20180530|05/30/2018 12:00:...|            NULL|                NULL|                 NULL|         NULL|
|      64667|               94110| 64667_20170814|08/14/2017 12:00:...|            NULL|                NULL|               

In [140]:
df_drop=prepared_df.dropna()
df_drop.show()

+-----------+--------------------+---------------+--------------------+----------------+--------------------+---------------------+-------------+
|business_id|business_postal_code|  inspection_id|     inspection_date|inspection_score|        violation_id|violation_description|risk_category|
+-----------+--------------------+---------------+--------------------+----------------+--------------------+---------------------+-------------+
|      97722|               94133| 97722_20181217|12/17/2018 12:00:...|              96|97722_20181217_10...| Unclean or degrad...|     Low Risk|
|      95023|               94112| 95023_20180510|05/10/2018 12:00:...|              92|95023_20180510_10...| Wiping cloths not...|     Low Risk|
|      94977|               94107| 94977_20180911|09/11/2018 12:00:...|              96|94977_20180911_10...| Inadequate and in...|Moderate Risk|
|      96578|               94108| 96578_20180817|08/17/2018 12:00:...|              78|96578_20180817_10...| Inadequately c

In [141]:
# Get the number of rows
num_rows = df_drop.count()

# Print the result
print(f"Number of rows in the DataFrame: {num_rows}")

Number of rows in the DataFrame: 36703


In [142]:
df_drop.printSchema()

root
 |-- business_id: integer (nullable = true)
 |-- business_postal_code: string (nullable = true)
 |-- inspection_id: string (nullable = true)
 |-- inspection_date: string (nullable = true)
 |-- inspection_score: integer (nullable = true)
 |-- violation_id: string (nullable = true)
 |-- violation_description: string (nullable = true)
 |-- risk_category: string (nullable = true)



Make sure to deal with date as month, day, year?

In [143]:
from pyspark.sql.functions import col, split, to_date, date_format

# Split the 'inspection_date' column by space to separate date and time, then take the date part
df_split = df_drop.withColumn("date", split(col("inspection_date"), " ").getItem(0))


# Show the resulting DataFrame
df_split.show(truncate=False)


+-----------+--------------------+---------------+----------------------+----------------+----------------------+------------------------------------------------------------------+-------------+----------+
|business_id|business_postal_code|inspection_id  |inspection_date       |inspection_score|violation_id          |violation_description                                             |risk_category|date      |
+-----------+--------------------+---------------+----------------------+----------------+----------------------+------------------------------------------------------------------+-------------+----------+
|97722      |94133               |97722_20181217 |12/17/2018 12:00:00 AM|96              |97722_20181217_103154 |Unclean or degraded floors walls or ceilings                      |Low Risk     |12/17/2018|
|95023      |94112               |95023_20180510 |05/10/2018 12:00:00 AM|92              |95023_20180510_103149 |Wiping cloths not clean or properly stored or inadequate saniti

In [144]:
# Drop the 'inspection_date' columns
df_split = df_split.drop("inspection_date")

# Show the resulting DataFrame
df_split.show(truncate=False)


+-----------+--------------------+---------------+----------------+----------------------+------------------------------------------------------------------+-------------+----------+
|business_id|business_postal_code|inspection_id  |inspection_score|violation_id          |violation_description                                             |risk_category|date      |
+-----------+--------------------+---------------+----------------+----------------------+------------------------------------------------------------------+-------------+----------+
|97722      |94133               |97722_20181217 |96              |97722_20181217_103154 |Unclean or degraded floors walls or ceilings                      |Low Risk     |12/17/2018|
|95023      |94112               |95023_20180510 |92              |95023_20180510_103149 |Wiping cloths not clean or properly stored or inadequate sanitizer|Low Risk     |05/10/2018|
|94977      |94107               |94977_20180911 |96              |94977_20180911_103

In [174]:
from pyspark.sql.functions import split, regexp_replace, col


from pyspark.sql.types import LongType  # Ensure you import the appropriate type

# Remove underscores and convert to LongType
df_split = df_split.withColumn("inspection_id", regexp_replace(col("inspection_id"), "_", ""))

# Show Results
df_split.show()

+-----------+--------------------+--------------+----------------+--------------------+---------------------+-------------+----------+
|business_id|business_postal_code| inspection_id|inspection_score|        violation_id|violation_description|risk_category|      date|
+-----------+--------------------+--------------+----------------+--------------------+---------------------+-------------+----------+
|      97722|               94133| 9772220181217|              96|97722_20181217_10...| Unclean or degrad...|     Low Risk|12/17/2018|
|      95023|               94112| 9502320180510|              92|95023_20180510_10...| Wiping cloths not...|     Low Risk|05/10/2018|
|      94977|               94107| 9497720180911|              96|94977_20180911_10...| Inadequate and in...|Moderate Risk|09/11/2018|
|      96578|               94108| 9657820180817|              78|96578_20180817_10...| Inadequately clea...|Moderate Risk|08/17/2018|
|      94928|               94121| 9492820190822|      

In [175]:
from pyspark.sql.functions import split, col, concat

# Split the 'violation_id' by underscore and take the first and last parts, then concatenate them without an underscore
df_cleaned = df_split.withColumn(
    "cleaned_violation_id", (split(col("violation_id"), "_").getItem(2))
)

# Show the results
df_cleaned.select("violation_id", "cleaned_violation_id", "violation_description").show(truncate=False)

+----------------------+--------------------+------------------------------------------------------------------+
|violation_id          |cleaned_violation_id|violation_description                                             |
+----------------------+--------------------+------------------------------------------------------------------+
|97722_20181217_103154 |103154              |Unclean or degraded floors walls or ceilings                      |
|95023_20180510_103149 |103149              |Wiping cloths not clean or properly stored or inadequate sanitizer|
|94977_20180911_103119 |103119              |Inadequate and inaccessible handwashing facilities                |
|96578_20180817_103124 |103124              |Inadequately cleaned or sanitized food contact surfaces           |
|94928_20190822_103150 |103150              |Improper or defective plumbing                                    |
|95401_20180607_103103 |103103              |High risk food holding temperature                 

In [176]:
# Drop the violation_id column
df_cleaned = df_cleaned.drop("violation_id")

# Show the resulting DataFrame
df_cleaned.show(truncate=False)

+-----------+--------------------+--------------+----------------+------------------------------------------------------------------+-------------+----------+--------------------+
|business_id|business_postal_code|inspection_id |inspection_score|violation_description                                             |risk_category|date      |cleaned_violation_id|
+-----------+--------------------+--------------+----------------+------------------------------------------------------------------+-------------+----------+--------------------+
|97722      |94133               |9772220181217 |96              |Unclean or degraded floors walls or ceilings                      |Low Risk     |12/17/2018|103154              |
|95023      |94112               |9502320180510 |92              |Wiping cloths not clean or properly stored or inadequate sanitizer|Low Risk     |05/10/2018|103149              |
|94977      |94107               |9497720180911 |96              |Inadequate and inaccessible handwa

In [177]:
# Get the number of rows
num_rows = df_cleaned.count()

# Print the result
print(f"Number of rows in the DataFrame: {num_rows}")

Number of rows in the DataFrame: 36703


In [178]:
# Iterate through each column to count the number of distinct values
for column in df_cleaned.columns:
    distinct_count = df_cleaned.select(column).distinct().count()
    print(f"Column '{column}' has {distinct_count} unique values.")


Column 'business_id' has 5168 unique values.
Column 'business_postal_code' has 51 unique values.
Column 'inspection_id' has 11666 unique values.
Column 'inspection_score' has 47 unique values.
Column 'violation_description' has 65 unique values.
Column 'risk_category' has 3 unique values.
Column 'date' has 770 unique values.
Column 'cleaned_violation_id' has 65 unique values.


In [179]:
df_cleaned.printSchema()

root
 |-- business_id: integer (nullable = true)
 |-- business_postal_code: string (nullable = true)
 |-- inspection_id: string (nullable = true)
 |-- inspection_score: integer (nullable = true)
 |-- violation_description: string (nullable = true)
 |-- risk_category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- cleaned_violation_id: string (nullable = true)



In [180]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Tokenizer, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, dayofweek, month, year, to_date

# Convert the date string to a proper date format
df_transformed = df_cleaned.withColumn("date_converted", to_date(col("date"), "MM/dd/yyyy"))

# Extract useful features like day_of_week, month, and year
df_transformed = df_transformed.withColumn("day_of_week", dayofweek(col("date_converted"))) \
                               .withColumn("month", month(col("date_converted"))) \
                               .withColumn("year", year(col("date_converted")))

# Optionally, you can also convert the date to an integer format in yyyyMMdd form
df_transformed = df_transformed.withColumn("date_int",
                                           (col("year") * 10000 + col("month") * 100 + col("day_of_week")).cast("int"))

# Business ID - Drop or use Label Encoding
indexer_business = StringIndexer(inputCol="business_id", outputCol="business_id_indexed")

# Business Postal Code - One-Hot Encoding or Label Encoding
indexer_postal_code = StringIndexer(inputCol="business_postal_code", outputCol="postal_code_indexed")

# Inspection ID - Drop the column (no predictive power)
df_transformed = df_transformed.drop("inspection_id")

# Inspection Score - Already numeric, cast to float
df_transformed = df_transformed.withColumn("inspection_score", df_transformed["inspection_score"].cast("float"))

# Inspection Type - Drop as only one value
df_transformed = df_transformed.drop("inspection_type")

# Cleaned Violation ID - Label Encoding or One-Hot Encoding
indexer_violation = StringIndexer(inputCol="cleaned_violation_id", outputCol="violation_id_indexed")

# Violation Description - Use TF-IDF or CountVectorizer
tokenizer = Tokenizer(inputCol="violation_description", outputCol="words")
hashing_tf = HashingTF(inputCol="words", outputCol="raw_features", numFeatures=10000)
idf = IDF(inputCol="raw_features", outputCol="features")

# Risk Category - Label Encoding (Ordinal Encoding)
indexer_risk = StringIndexer(inputCol="risk_category", outputCol="risk_category_indexed")

# Create a Pipeline for Feature Transformation
pipeline = Pipeline(stages=[
    indexer_business,
    indexer_postal_code,
    indexer_violation,
    indexer_risk,
    tokenizer,
    hashing_tf,
    idf
])

# Fit and transform the pipeline
df_transformed = pipeline.fit(df_transformed).transform(df_transformed)

# Show the final transformed DataFrame, including the features and the new extracted columns
df_transformed.select("business_id_indexed", "postal_code_indexed", "violation_id_indexed",
                      "features", "risk_category_indexed", "day_of_week", "month", "inspection_score").show(truncate=False)


+-------------------+-------------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+-----------+-----+----------------+
|business_id_indexed|postal_code_indexed|violation_id_indexed|features                                                                                                                                                                                                                   |risk_category_indexed|day_of_week|month|inspection_score|
+-------------------+-------------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+-----------+----

In [181]:
df_transformed.printSchema()

root
 |-- business_id: integer (nullable = true)
 |-- business_postal_code: string (nullable = true)
 |-- inspection_score: float (nullable = true)
 |-- violation_description: string (nullable = true)
 |-- risk_category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- cleaned_violation_id: string (nullable = true)
 |-- date_converted: date (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- date_int: integer (nullable = true)
 |-- business_id_indexed: double (nullable = false)
 |-- postal_code_indexed: double (nullable = false)
 |-- violation_id_indexed: double (nullable = false)
 |-- risk_category_indexed: double (nullable = false)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)



In [182]:
df_cleaned.write.csv("/content/data_cleaned(2).csv", header=True, mode="overwrite")


In [183]:
!zip -r /content/cleaned_restaurant_inspections_single.zip /content/cleaned_restaurant_inspections_single
from google.colab import files
files.download('/content/cleaned_restaurant_inspections_single.zip')



	zip warning: name not matched: /content/cleaned_restaurant_inspections_single

zip error: Nothing to do! (try: zip -r /content/cleaned_restaurant_inspections_single.zip . -i /content/cleaned_restaurant_inspections_single)


FileNotFoundError: Cannot find file: /content/cleaned_restaurant_inspections_single.zip

In [162]:
df_transformed.write.csv("/content/data_transformed.csv", header=True, mode="overwrite")

AnalysisException: [UNSUPPORTED_DATA_TYPE_FOR_DATASOURCE] The CSV datasource doesn't support the column `words` of the type "ARRAY<STRING>".

In [184]:
# Convert Spark DataFrame to Pandas
pd_transformed_df = df_transformed.toPandas()

pd_transformed_df.head()

,business_id,business_postal_code,inspection_score,violation_description,risk_category,date,cleaned_violation_id,date_converted,day_of_week,month,year,date_int,business_id_indexed,postal_code_indexed,violation_id_indexed,risk_category_indexed,words,raw_features,features
0,97722,94133,96.0,Unclean or degraded floors walls or ceilings,Low Risk,12/17/2018,103154,2018-12-17,2,12,2018,20181202,4702.0,4.0,0.0,0.0,"[unclean, or, degraded, floors, walls, or, cei...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,95023,94112,92.0,Wiping cloths not clean or properly stored or ...,Low Risk,05/10/2018,103149,2018-05-10,5,5,2018,20180505,3589.0,7.0,5.0,0.0,"[wiping, cloths, not, clean, or, properly, sto...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,94977,94107,96.0,Inadequate and inaccessible handwashing facili...,Moderate Risk,09/11/2018,103119,2018-09-11,3,9,2018,20180903,5075.0,6.0,3.0,1.0,"[inadequate, and, inaccessible, handwashing, f...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,96578,94108,78.0,Inadequately cleaned or sanitized food contact...,Moderate Risk,08/17/2018,103124,2018-08-17,6,8,2018,20180806,2304.0,9.0,2.0,1.0,"[inadequately, cleaned, or, sanitized, food, c...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,94928,94121,90.0,Improper or defective plumbing,Low Risk,08/22/2019,103150,2019-08-22,5,8,2019,20190805,1371.0,11.0,20.0,0.0,"[improper, or, defective, plumbing]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [185]:
pd_transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36703 entries, 0 to 36702
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   business_id            36703 non-null  int32  
 1   business_postal_code   36703 non-null  object 
 2   inspection_score       36703 non-null  float32
 3   violation_description  36703 non-null  object 
 4   risk_category          36703 non-null  object 
 5   date                   36703 non-null  object 
 6   cleaned_violation_id   36703 non-null  object 
 7   date_converted         36703 non-null  object 
 8   day_of_week            36703 non-null  int32  
 9   month                  36703 non-null  int32  
 10  year                   36703 non-null  int32  
 11  date_int               36703 non-null  int32  
 12  business_id_indexed    36703 non-null  float64
 13  postal_code_indexed    36703 non-null  float64
 14  violation_id_indexed   36703 non-null  float64
 15  ri

In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, to_date, to_timestamp, isnotnull

# # Initialize Spark Session
# spark = SparkSession.builder.appName("ChangeColumnTypes").getOrCreate()

# # Set legacy time parser policy
# spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# # Convert Spark DataFrame back to Pandas DataFrame if needed
# # pandas_df = df_cleaned.toPandas()

# # Create a Spark DataFrame from the Pandas DataFrame
# spark_df = spark.createDataFrame(pandas_df)

# # Now apply withColumn to the Spark DataFrame (spark_df)
# df_cleaned = spark_df.withColumn("business_id", col("business_id").cast("int")) \
#                .withColumn("inspection_id", col("inspection_id").cast("int")) \
#                .withColumn("inspection_score", col("inspection_score").cast("int")) \
#                .withColumn("violation_id", col("violation_id").cast("int")) \
#                .withColumn("business_postal_code", col("business_postal_code").cast("int")) \
#                .withColumn("inspection_date", to_date(col("inspection_date"), "MM/dd/yyyy hh:mm:ss a")) \
#                .withColumn("date", to_date(col("date"), "MM/dd/yyyy")) \
#                .withColumn("time", to_timestamp(col("time"), "HH:mm:ss"))  # Convert time to timestamp

# # Show Schema After Conversion
# df_cleaned.printSchema()

In [ ]:
# import pandas as pd

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder

# # Drop unnecessary columns
# columns_to_drop = ['business_name', 'business_address', 'inspection_type']
# # 'date' and 'time' are dropped here
# df = pandas_df.drop(columns=columns_to_drop, errors='ignore')

# # Handle categorical variables
# categorical_cols = df.select_dtypes(include='object').columns
# df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# # Check
# df_encoded.head()

# **REGRESSION**

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression, LogisticRegression
# from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# from sklearn.svm import SVR, SVC
# from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, classification_report

# # 6. Features & Target for Regression
# X_reg = df_encoded.drop('inspection_score', axis=1)
# y_reg = df_encoded['inspection_score']

# # Convert datetime columns to numerical representation (e.g., Unix timestamp)
# for col in X_reg.select_dtypes(include=['datetime64']).columns:
#     X_reg[col] = X_reg[col].astype(np.int64) // 10**9  # Convert to Unix timestamp

# # 7. Train-Test Split
# X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# # 8. Apply StandardScaler (only for Linear Regression and SVM)
# scaler = StandardScaler()
# X_train_reg_scaled = scaler.fit_transform(X_train_reg)
# X_test_reg_scaled = scaler.transform(X_test_reg)

# # 9. Train Models
# # Linear Regression (Scaled)
# lr = LinearRegression()
# lr.fit(X_train_reg_scaled, y_train_reg)
# y_pred_lr = lr.predict(X_test_reg_scaled)

# # Random Forest Regressor (No Scaling Needed)
# rf_reg = RandomForestRegressor()
# rf_reg.fit(X_train_reg, y_train_reg)
# y_pred_rf = rf_reg.predict(X_test_reg)

# # SVR (Scaled)
# svr = SVR()
# svr.fit(X_train_reg_scaled, y_train_reg)
# y_pred_svr = svr.predict(X_test_reg_scaled)

# # 10. Evaluate Regression Models
# rmse_lr = mean_squared_error(y_test_reg, y_pred_lr) ** 0.5  # Calculate RMSE manually
# rmse_rf = mean_squared_error(y_test_reg, y_pred_rf) ** 0.5  # Calculate RMSE manually
# rmse_svr = mean_squared_error(y_test_reg, y_pred_svr) ** 0.5  # Calculate RMSE manually


# print(f"Linear Regression RMSE: {rmse_lr}")
# print(f"Random Forest Regressor RMSE: {rmse_rf}")
# print(f"SVR RMSE: {rmse_svr}")



In [186]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Step 1: Encode categorical features
# Label encode columns with few categories (e.g., 'risk_category')
label_encoder = LabelEncoder()
pd_transformed_df['risk_category_encoded'] = label_encoder.fit_transform(pd_transformed_df['risk_category'])

# One-Hot encode columns with many categories (e.g., 'business_postal_code', 'violation_description')
df_encoded = pd.get_dummies(df_encoded, columns=['business_postal_code', 'violation_description'], drop_first=True)

# Step 2: Handle date columns (if needed)
# You can either drop the 'date' or 'date_converted' columns or extract time features:
df_encoded['date_converted'] = pd.to_datetime(df_encoded['date_converted'])
df_encoded['year'] = df_encoded['date_converted'].dt.year
df_encoded['month'] = df_encoded['date_converted'].dt.month
df_encoded['day'] = df_encoded['date_converted'].dt.day
df_encoded.drop(['date_converted', 'date'], axis=1, inplace=True)

# Step 3: Separate features and target
X = df_encoded.drop(['inspection_score'], axis=1)
y = df_encoded['inspection_score']

# Step 4: Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Apply scaling to numerical columns
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=[np.number]))
X_test_scaled = scaler.transform(X_test.select_dtypes(include=[np.number]))

# Step 6: Train models (example with Linear Regression)
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

# Step 7: Evaluate the model
from sklearn.metrics import mean_squared_error
rmse_lr = mean_squared_error(y_test, y_pred_lr) ** 0.5
print(f"Linear Regression RMSE: {rmse_lr}")


Linear Regression RMSE: 6.36326362709294


In [187]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, classification_report

# 6. Features & Target for Regression
X_reg = df_encoded.drop('inspection_score', axis=1)
y_reg = df_encoded['inspection_score']

# Convert datetime columns to numerical representation (e.g., Unix timestamp)
for col in X_reg.select_dtypes(include=['datetime64']).columns:
    X_reg[col] = X_reg[col].astype(np.int64) // 10**9  # Convert to Unix timestamp

# 7. Train-Test Split
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# 8. Apply StandardScaler (only for Linear Regression and SVM)
scaler = StandardScaler()
X_train_reg_scaled = scaler.fit_transform(X_train_reg)
X_test_reg_scaled = scaler.transform(X_test_reg)

# 9. Train Models
# Linear Regression (Scaled)
lr = LinearRegression()
lr.fit(X_train_reg_scaled, y_train_reg)
y_pred_lr = lr.predict(X_test_reg_scaled)

# Random Forest Regressor (No Scaling Needed)
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train_reg, y_train_reg)
y_pred_rf = rf_reg.predict(X_test_reg)

# SVR (Scaled)
svr = SVR()
svr.fit(X_train_reg_scaled, y_train_reg)
y_pred_svr = svr.predict(X_test_reg_scaled)

# 10. Evaluate Regression Models
rmse_lr = mean_squared_error(y_test_reg, y_pred_lr) ** 0.5  # Calculate RMSE manually
rmse_rf = mean_squared_error(y_test_reg, y_pred_rf) ** 0.5  # Calculate RMSE manually
rmse_svr = mean_squared_error(y_test_reg, y_pred_svr) ** 0.5  # Calculate RMSE manually


print(f"Linear Regression RMSE: {rmse_lr}")
print(f"Random Forest Regressor RMSE: {rmse_rf}")
print(f"SVR RMSE: {rmse_svr}")


ValueError: could not convert string to float: 'Moderate Risk'

In [ ]:
# 10. Line Chart: Regression Model Comparison
model_names_reg = ['Linear Regression', 'Random Forest', 'SVR']
rmse_scores = [rmse_lr, rmse_rf, rmse_svr]

plt.figure(figsize=(8,5))
plt.plot(model_names_reg, rmse_scores, marker='o', color='purple')
plt.title('Regression Models RMSE Comparison')
plt.xlabel('Models')
plt.ylabel('RMSE')
plt.grid(True)
plt.show()

In [ ]:
print("📊 Report: Regression Models RMSE Analysis")
print("The Random Forest Regressor has the lowest RMSE, meaning it's the best model for predicting inspection scores. The SVR model performs the worst, confirming that tree-based methods work better for this dataset.")


# **CLASSIFICATION**

In [ ]:
# 12. Features & Target for Classification
target_col = [col for col in df_encoded.columns if 'risk_category' in col][0]
X_clf = df_encoded.drop(target_col, axis=1)
y_clf = df_encoded[target_col]

# Convert datetime columns to numerical representation (e.g., Unix timestamp)
for col in X_clf.select_dtypes(include=['datetime64']).columns:
    X_clf[col] = X_clf[col].astype(np.int64) // 10**9  # Convert to Unix timestamp


# 13. Train-Test Split
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

# 14. Apply StandardScaler (only for Logistic Regression and SVM)
X_train_clf_scaled = scaler.fit_transform(X_train_clf)
X_test_clf_scaled = scaler.transform(X_test_clf)

# 15. Train Classification Models
# Logistic Regression (Scaled)
logr = LogisticRegression(max_iter=1000)
logr.fit(X_train_clf_scaled, y_train_clf)
y_pred_logr = logr.predict(X_test_clf_scaled)

# Random Forest Classifier (No Scaling Needed)
rfc = RandomForestClassifier()
rfc.fit(X_train_clf, y_train_clf)
y_pred_rfc = rfc.predict(X_test_clf)

# SVM Classifier (Scaled)
svc = SVC()
svc.fit(X_train_clf_scaled, y_train_clf)
y_pred_svc = svc.predict(X_test_clf_scaled)

# 16. Evaluate Classification Models
acc_logr = accuracy_score(y_test_clf, y_pred_logr)
acc_rfc = accuracy_score(y_test_clf, y_pred_rfc)
acc_svc = accuracy_score(y_test_clf, y_pred_svc)

print(f"Logistic Regression Accuracy: {acc_logr}")
print(f"Random Forest Classifier Accuracy: {acc_rfc}")
print(f"SVM Classifier Accuracy: {acc_svc}")



In [ ]:
# 17. Line Chart: Classification Accuracy Comparison
model_names_clf = ['Logistic Regression', 'Random Forest', 'SVM']
accuracy_scores = [acc_logr, acc_rfc, acc_svc]

plt.figure(figsize=(8,5))
plt.plot(model_names_clf, accuracy_scores, marker='s', linestyle='-', color='green')
plt.title('Classification Models Accuracy Comparison')
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.grid(True)
plt.show()

In [ ]:
print("📊 Report: Classification Models Accuracy Analysis")
print("Random Forest Classifier achieves the highest accuracy, proving it's the best model for risk classification. Logistic Regression performs decently, while SVM has slightly lower accuracy.")


In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

# Assuming risk_category column exists
# Check column names first
print(pandas_df.columns)

# Let's say column name is 'risk_category_Low' or similar after encoding
target_col = [col for col in df_encoded.columns if 'risk_category' in col][0]

# Features and Target
X_class = df_encoded.drop(target_col, axis=1)
y_class = df_encoded[target_col]

# Convert datetime columns to numerical representation (e.g., Unix timestamp)
for col in X_class.select_dtypes(include=['datetime64']).columns:
    X_class[col] = X_class[col].astype(np.int64) // 10**9  # Convert to Unix timestamp

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Apply StandardScaler (only for Logistic Regression and SVM)
scaler = StandardScaler() # Initialize StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression
logr = LogisticRegression(max_iter=1000)
logr.fit(X_train_scaled, y_train)  # Use scaled data for Logistic Regression
y_pred_logr = logr.predict(X_test_scaled)  # Use scaled data for prediction

# Random Forest Classifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)

# SVM Classifier
svc = SVC()
svc.fit(X_train_scaled, y_train)  # Use scaled data for SVM
y_pred_svc = svc.predict(X_test_scaled)  # Use scaled data for prediction

# Evaluation
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_logr))
print(confusion_matrix(y_test, y_pred_logr))
print(classification_report(y_test, y_pred_logr))

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rfc))
print(confusion_matrix(y_test, y_pred_rfc))
print(classification_report(y_test, y_pred_rfc))

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svc))
print(confusion_matrix(y_test, y_pred_svc))
print(classification_report(y_test, y_pred_svc))

# **Confusion Matrix (Classification Results)**

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

# For Random Forest Classifier
ConfusionMatrixDisplay.from_estimator(rfc, X_test, y_test, cmap='Blues')
plt.title('Random Forest Classifier - Confusion Matrix')
plt.show()

# Similarly, for Logistic Regression:
ConfusionMatrixDisplay.from_estimator(logr, X_test, y_test, cmap='Greens')
plt.title('Logistic Regression - Confusion Matrix')
plt.show()

**The confusion matrix **for the Random Forest Classifier shows the number of correct and incorrect predictions for each risk category.
The diagonal values (top-left to bottom-right) indicate correct classifications (True Positives and True Negatives).
The off-diagonal values indicate misclassifications where the model predicted the wrong category.
If most values are concentrated along the diagonal, it means the model is performing well.

**The confusion matrix for Logistic Regression** follows the same structure as above.
Since Logistic Regression is a linear model, it may perform worse if the data is not linearly separable.
If Logistic Regression's confusion matrix has more misclassifications (compared to Random Forest), it indicates that the problem is likely non-linear, and tree-based models (like Random Forest) are a better fit.

# **Feature vs. Target (Trend Lines)**

In [ ]:
# Plot using business_id (or another existing column) instead of inspection_id
plt.figure(figsize=(10, 5))
# Assuming 'inspection_score' is present in pandas_df
plt.scatter(pandas_df['business_id'], pandas_df['inspection_score'], marker='o', linestyle='-')
plt.title('Inspection Scores Across Different Businesses')
plt.xlabel('Business ID')
plt.ylabel('Inspection Score')
plt.show()





# **Regression: Actual vs Predicted Plot**

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_rf, color='blue', label='Predicted vs Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3, label='Perfect Prediction')
plt.xlabel('Actual Inspection Score')
plt.ylabel('Predicted Inspection Score')
plt.legend()
plt.title('Random Forest Regressor: Actual vs Predicted Scores')
plt.show()


# **Classification Report as Heatmap**

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

# Generate classification report
report = classification_report(y_test, y_pred_rfc, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(df_report.iloc[:-1, :-1], annot=True, cmap='coolwarm')
plt.title('Random Forest Classifier - Classification Report')
plt.show()
